In [6]:
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import os
import time

In [7]:
def connect_mongodb(uri,db_name, collection_name):
    # Connect to MongoDB 
    client = MongoClient(uri)
    
    # Select the database and collection
    db = client[db_name]
    collection = db[collection_name]
    
    return collection
def query_mongodb(collection, query, fields=None):
    """
    Queries the MongoDB collection and returns specific fields or all fields if no specific ones are provided.
    
    Args:
        collection: The MongoDB collection object.
        query (dict): The query to filter documents.
        fields (list): List of fields to return from the query. If None, all fields are returned.
    
    Returns:
        result: List of documents with the specified fields or all fields.
    """
    # If fields is None, return all fields
    if fields:
        # Return only the specified fields
        projection = {field: 1 for field in fields}
    else:
        # Return all fields
        projection = None
    
    # Perform the query
    result = collection.find(query, projection)
    
    # Convert the cursor to a list and return
    return list(result)
def update_mongo_fields(collection, query_filter, update_fields):
    """
    Update one or more fields in a MongoDB collection.

    Parameters:
    - collection: The MongoDB collection object.
    - query_filter: A dictionary specifying the filter to select documents.
    - update_fields: A dictionary of fields to be updated, with keys as field names and values as the new values.

    Example:
    update_mongo_fields(
        collection=your_collection,
        query_filter={'_id': 'in.codeseed.audify'},
        update_fields={'apkfile_name': 'new_name.apk', 'extract_manifest': 'UPDATED'}
    )
    """
    # Print query filter for debugging
    print(f"Query filter: {query_filter}")

    # Prepare the update object with $set to update only the specified fields
    update_query = {"$set": update_fields}

    # Perform the update in the collection
    result = collection.update_many(query_filter, update_query)

    # Check if anything was matched or updated
    if result.matched_count == 0:
        print("No documents matched the filter.")
    if result.modified_count == 0 and result.matched_count > 0:
        print("Documents matched but none were updated (maybe no fields were changed).")
    
    # Print out the result of the update
    print(f"Matched {result.matched_count} document(s), updated {result.modified_count} document(s).")

def check_value_in_array(value, array):
    if value in array:
        return 1
    else:
        return 0

In [8]:
mongoDB_uri = 'mongodb://localhost:27017'
mongoDB_database = 'wearable-project' 
# mongoDB_collection = 'wearable-app'
# standalone
mongoDB_collection = 'wearable-standalone'

In [9]:
# 1. Setup selenium
chromedriver = 'chromedriver.exe'
os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument('--no-proxy-server')
# Prevent first-time setup and use a fresh Chrome profile
chrome_options.add_argument("user-data-dir=/tmp/selenium")
chrome_options.add_argument("--no-first-run")
chrome_options.add_argument("--no-default-browser-check")
chrome_options.add_argument("--disable-search-engine-choice-screen")
prefs = {
    'profile.default_content_setting_values.automatic_downloads': 1,
    "download.default_directory": "",
    'profile.default_content_setting_values.popups': 0,
    'profile.default_content_setting_values.notifications': 2,
    "profile.default_content_setting_values.geolocation": 2,
    "profile.default_content_setting_values.media_stream": 2,
    "profile.default_content_setting_values.cookies": 2,
    "profile.default_content_setting_values.javascript": 1,
    "profile.default_content_setting_values.plugins": 1,
    "profile.default_content_setting_values.fullscreen": 2,
    'profile.default_content_setting_values.automatic_downloads': 1,
    'profile.default_content_settings.popups': 0,
    'profile.default_content_setting_values.default_search_engine_id': 1,
    'profile.default_content_setting_values.prompt_for_download': False,
    'profile': {'default_content_setting_values': {'search_suggest_enabled': False}},
    'intl.accept_languages': 'en,en_US',
    'first_run': False,
    'browser.startup.page': 0,
    'browser.startup.homepage': 'about:blank',
}
chrome_options.add_experimental_option("prefs", prefs)

In [10]:
# Connect to the MongoDB collection
collection = connect_mongodb(mongoDB_uri,mongoDB_database,mongoDB_collection)
# Specify the fields you want to return (e.g., '_id', 'apkfile_name')
fields = ['_id']

# Fetch the specified fields only
result = query_mongodb(collection,None, fields)

# Print the result (this will contain only '_id' and 'apkfile_name')
#print(result)
# 0 - 884
for i in range(len(result)):
    print("-----------------Loop-"+str(i)+"-----------------")
    pkg_name = result[i]['_id']
    print("pkg_name: ",pkg_name)
    web = webdriver.Chrome(options=chrome_options)
    web.maximize_window()
    apk_path = "https://play.google.com/store/apps/details?id="+pkg_name
    print("apk_path",apk_path)
    web.get(apk_path)
    time.sleep(2)
    #
    try:
        error_element = web.find_element(By.ID, "error-section")
        error_text = error_element.text
        if(error_text=="We're sorry, the requested URL was not found on this server."):
            print("APK not found")
            update_mongo_fields(
                    collection, 
                    {'_id': str(pkg_name)}, 
                    {'ggplaystore': 'Not Found'}
                )
            web.close()
            continue
    except NoSuchElementException:
        support_lists=web.find_elements(By.CSS_SELECTOR,"span[class='ypTNYd']")
        print(len(support_lists))
        if len(support_lists)>0:
            support_array = []
            for i in range(len(support_lists)):
                support_name=web.find_elements(By.CSS_SELECTOR,"span[class='ypTNYd']")[i].text
                support_array.append(support_name)
            print("support_array:", support_array)
            if(check_value_in_array('Watch', support_array)==1):
                update_mongo_fields(
                    collection, 
                    {'_id': str(pkg_name)}, 
                    {'ggplaystore': 'Yes'}
                )
            else:
                update_mongo_fields(
                    collection, 
                    {'_id': str(pkg_name)}, 
                    {'ggplaystore': 'No'}
                )
            web.close()
        else:
            update_mongo_fields(
                    collection, 
                    {'_id': str(pkg_name)}, 
                    {'ggplaystore': 'No'}
                )
            web.close()
    #break

-----------------Loop-0-----------------
pkg_name:  app.groupcal.www
apk_path https://play.google.com/store/apps/details?id=app.groupcal.www
8
support_array: ['Phone', 'Tablet', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']
Query filter: {'_id': 'app.groupcal.www'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-1-----------------
pkg_name:  apps.r.compass
apk_path https://play.google.com/store/apps/details?id=apps.r.compass
9
support_array: ['Phone', 'Watch', 'Tablet', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']
Query filter: {'_id': 'apps.r.compass'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-2-----------------
pkg_name:  ch.publisheria.bring
apk_path https://play.google.com/store/apps/details?id=ch.publisheria.bring
10
support_array: ['Phone', 'Watch', 'Chromebook', 'Tablet', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']
Query filter: {'_id': 'ch.publisheria.bring'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-3-----------------
pkg_n

10
support_array: ['Phone', 'Watch', 'Car', 'Tablet', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']
Query filter: {'_id': 'com.google.android.apps.maps'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-26-----------------
pkg_name:  com.google.android.apps.youtube.music
apk_path https://play.google.com/store/apps/details?id=com.google.android.apps.youtube.music
11
support_array: ['Phone', 'Watch', 'Car', 'Chromebook', 'Tablet', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']
Query filter: {'_id': 'com.google.android.apps.youtube.music'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-27-----------------
pkg_name:  com.google.android.contacts
apk_path https://play.google.com/store/apps/details?id=com.google.android.contacts
10
support_array: ['Phone', 'Watch', 'Chromebook', 'Tablet', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']
Query filter: {'_id': 'com.google.android.contacts'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-28-----------------
pk

apk_path https://play.google.com/store/apps/details?id=com.shazam.android
10
support_array: ['Phone', 'Watch', 'Chromebook', 'Tablet', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']
Query filter: {'_id': 'com.shazam.android'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-51-----------------
pkg_name:  com.shotzoom.golfshot2
apk_path https://play.google.com/store/apps/details?id=com.shotzoom.golfshot2
9
support_array: ['Phone', 'Watch', 'Tablet', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']
Query filter: {'_id': 'com.shotzoom.golfshot2'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-52-----------------
pkg_name:  com.skimble.workouts
apk_path https://play.google.com/store/apps/details?id=com.skimble.workouts
10
support_array: ['Phone', 'Watch', 'Chromebook', 'Tablet', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No']
Query filter: {'_id': 'com.skimble.workouts'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-53-----------------
pkg_name:  com.soun